# CAD Processing

## Installation

The ABC dataset offers 1 Million CAD files and a specialized library to process the files and generate arbitrary dense meshes and point clouds. The library can be installed with `conda install -c pythonocc -c oce -c dlr-sc -c conda-forge -c tpaviot -c skoch9 cadmesh`. 

## Import

In [ ]:
import cadmesh as cm
import meshplot as mp
import numpy as np
import igl

## Meshing CAD files

In [ ]:
m_coarse = cm.mesh_model("data/test1.step", max_size=4e-5)

In [ ]:
mp.plot(m_coarse["vertices"], m_coarse["face_indices"], shading={"wireframe": True, "wire_width": 1.0})

In [ ]:
m_dense = cm.mesh_model("data/test1.step", max_size=2e-5)
mp.plot(m_dense["vertices"], m_dense["face_indices"], shading={"wireframe": True, "wire_width": 1.0})

## Ground Truth Quantities

In [ ]:
from utils import read_model

m = read_model("data/test1_trimesh.obj", "data/test1_features.yml")

# Average normals at vertices with multiple normals
av_normals = cm.get_averaged_normals(m)

# Determine normals with uniform weighting in libigl
normals = igl.per_vertex_normals(m["vertices"], m["face_indices"].astype("int64"))

# Plot the model
p = mp.plot(m["vertices"], m["face_indices"], return_plot=True)

# Add normals to the plot
p.add_lines(m["vertices"], m["vertices"] + normals * 2.0, shading={"line_color": "red"});
p.add_lines(m["vertices"], m["vertices"] + av_normals * 2.0, shading={"line_color": "black"})

## Feature Curves and Patches

In [ ]:
# Retrieve the surface patches
c = np.zeros(m["face_indices"].shape[0])
for i, fe in enumerate(m["features"]["surfaces"]):
    for j in fe["face_indices"]:
        c[j] = i

# Visualize the patches
mp.plot(m["vertices"], m["face_indices"], c)

In [ ]:
# Retrieve the surface patch types
t_map = {"Plane": 0, "Cylinder": 1, "Cone": 2, "Sphere": 3, "Torus": 4, "Bezier": 5,
         "BSpline": 6, "Revolution": 7,"Extrusion": 8, "Other": 9}

c1 = np.zeros(m["face_indices"].shape[0])
for i, fe in enumerate(m["features"]["surfaces"]):
    t = t_map[fe["type"]]
    for j in fe["face_indices"]:
        c1[j] = t

# Visualize the patch types
mp.plot(m["vertices"], m["face_indices"], c1)

In [ ]:
# Retrieve the surface patch vertices
c2 = np.zeros(m["vertices"].shape[0])
for i, fe in enumerate(m["features"]["surfaces"]):
    for j in fe["vert_indices"]:
        c2[j] = i

# Visualize the vertices
mp.plot(m["vertices"], c=c2, shading={"point_size": 10.})

In [ ]:
# Retrieve the sharp features
lines = []
for i, fe in enumerate(m["features"]["curves"]):
    if fe["sharp"]:
        for j in range(len(fe["vert_indices"])-1):
            lines.append([fe["vert_indices"][j], fe["vert_indices"][j+1]])        

# Visualize the sharp features            
p = mp.plot(m["vertices"], m["face_indices"], c, return_plot=True)
p.add_edges(m["vertices"], np.array(lines), shading={"line_color": "red", "line_width": 2.5})

# Machine Learning

## Installation

For the learning examples we use [Pytorch](https://pytorch.org/) and [Pytorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) that can be installed according to their documentation.

## Import

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Sequential as Seq, Dropout, Linear as Lin
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import DynamicEdgeConv

from utils import MLP
from utils import ABCDataset
import meshplot as mp
import numpy as np

## Data Loading and Visualization

In [ ]:
tf_train = T.Compose([
    T.FixedPoints(100),
    T.RandomTranslate(0.002),
    T.RandomRotate(15, axis=0),
    T.RandomRotate(15, axis=1),
    T.RandomRotate(15, axis=2)
])
tf_test = T.Compose([
    T.FixedPoints(2000)
])
tf_pre = T.NormalizeScale()

typ = "Curves"
train_dataset = ABCDataset("data/ml/ABC", train=True, typ=typ, transform=tf_train, pre_transform=tf_pre)
test_dataset = ABCDataset("data/ml/ABC", train=False, typ=typ, transform=tf_test, pre_transform=tf_pre)

train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, num_workers=6)

In [ ]:
dataset = test_dataset
#print(len(dataset), dataset.num_classes, dataset.num_node_features)

d = dataset[4]
v = d.pos.cpu().numpy()
y = d.y.cpu().numpy()

if typ == "Curves":
    p = mp.plot(v, c=-y, shading={"point_size": 0.15})

if typ == "Normals":
    c = y * 0.5 + 0.5
    c = np.linalg.norm(c, axis=1)
    p = mp.plot(v, c=c, shading={"point_size": 0.15}, return_plot=True)

    p.add_lines(v, v + y * 0.05)
p

## Initializing the Network

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, out_channels, k=30, aggr='max', typ='Curves'):
        super(Net, self).__init__()

        self.conv1 = DynamicEdgeConv(MLP([2 * 3, 64, 64]), k, aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)
        self.conv3 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)
        self.lin1 = MLP([3 * 64, 1024])

        self.mlp = Seq(
            MLP([1024, 256]), Dropout(0.5), MLP([256, 128]), Dropout(0.5),
            Lin(128, out_channels))

    def forward(self, data):
        pos, batch = data.pos, data.batch
        x1 = self.conv1(pos, batch)
        x2 = self.conv2(x1, batch)
        x3 = self.conv3(x2, batch)
        out = self.lin1(torch.cat([x1, x2, x3], dim=1))
        out = self.mlp(out)
        if typ == "Curves":
            return F.log_softmax(out, dim=1)
        if typ == "Normals":
            return F.normalize(out, p=2, dim=-1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(train_dataset.num_classes, k=30).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.8)

## Defining the Train and Test Procedures

In [ ]:
class Cosine_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Cosine_Loss,self).__init__()
        
    def forward(self, x, y):
        dotp = torch.mul(x, y).sum(1)
        angle = torch.sum(torch.acos(torch.abs(dotp))) / x.shape[0]
        loss = torch.sum(1 - dotp.pow(2)) / x.shape[0]
        return loss, angle

cosine_loss = Cosine_Loss()


def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        if typ == "Curves":
            loss = F.nll_loss(out, data.y)
        if typ == "Normals":
            loss, angle = cosine_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if typ == "Curves":
            correct_nodes += out.max(dim=1)[1].eq(data.y).sum().item()
            total_nodes += data.num_nodes
            acc = correct_nodes / total_nodes
        if typ == "Normals":
            acc = angle.item()*180/np.pi
        
        print('[{}/{}] Loss: {:.4f}, Train Accuracy: {:.4f}'.format(i + 1, len(train_loader), total_loss / 10, acc))
            
        total_loss = correct_nodes = total_nodes = 0


def test(loader):
    model.eval()

    correct_nodes = total_nodes = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
            
        if typ == "Curves":
            pred = out.max(dim=1)[1]
            correct_nodes += pred.eq(data.y).sum().item()
            total_nodes += data.num_nodes
            
        if typ == "Normals":
            _, angle = cosine_loss(out, data.y)
            correct_nodes += angle.item() * 180 / np.pi
            total_nodes += 1
            
    return correct_nodes / total_nodes

## Running the Training and Saving the Model

In [ ]:
for epoch in range(1, 2):
    train()
    acc = test(test_loader)
    print('Epoch: {:02d}, Accuracy: {:.4f}'.format(epoch, acc))
    torch.save(model.state_dict(), "data/ml/ABC/models/%02i_%.2f.dat"%(epoch, acc))

## Loading a Model

In [ ]:
model = Net(train_dataset.num_classes, k=30)
model.load_state_dict(torch.load("data/ml/ABC/models/m_022_0.9606.dat", map_location='cpu'));

## Result Visualization

In [ ]:
loader = test_loader

for d in loader:
    with torch.no_grad():
        out = model(d.to(device))
    
    m = 4
    v = d.pos[d.batch == m].cpu().numpy()
    y = d.y[d.batch == m].cpu().numpy()
    
    
    if typ == "Curves":
        e = out.max(dim=1)[1][d.batch == m].cpu().numpy()
        p = mp.subplot(v, c=-y, shading={"point_size": 0.15}, s=[1, 2, 0])
        mp.subplot(v, c=-e, shading={"point_size": 0.15}, s=[1, 2, 1], data=p)

    if typ == "Normals":
        n = out[d.batch == m].cpu().numpy()
        nc = n * 0.5 + 0.5
        nc = np.linalg.norm(nc, axis=1)
        c = y * 0.5 + 0.5
        c = np.linalg.norm(c, axis=1)
        p = mp.subplot(v, c=-c, shading={"point_size": 0.15}, s=[1, 2, 0])
        mp.subplot(v, c=-nc, shading={"point_size": 0.15}, s=[1, 2, 1], data=p)

        p.rows[0][0].add_lines(v, v + y * 0.05)
        p.rows[0][1].add_lines(v, v + n * 0.05)
    break
p

In [ ]:
acc = test(test_loader)
print('Accuracy: {:.4f}'.format(acc))